In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
import geojson

In [ ]:
# helper function to get  each women's time mandate_end as a column
def get_last_year_in_office(df):
    functions = {'hrllo': (' (assassinated)','',regex=False), 
                    'test': ('Incumbent',np.NaNm),
                     fillna: df['Mandate start']}


In [22]:
# read in female heads of state dataset
df = pd.read_csv('data/female_heads_of_state.csv')
countries = pd.read_csv('data/list_of_countries.csv')

In [24]:
# add in the Alpha-3 code
df = df.merge(countries[['Country','Alpha-3 code']],on='Country')

# add colunms that have the start and end years of their terms
df['start_year'] = df['Mandate start'].apply(lambda x: f'19{x[-2:]}' if int(x[-2:]) > 39 else f'20{x[-2:]}')
df['Mandate end'] = df['Mandate end'].str.replace(' (assassinated)','',regex=False)
df['Mandate end'] = df['Mandate end'].replace('Incumbent',np.NaN)
df['Mandate end'] = df['Mandate end'].fillna(df['Mandate start'])
df['end_year'] = df['Mandate end'].apply(lambda x: f'19{x[-2:]}' if int(x[-2:]) > 39 else f'20{x[-2:]}')

In [25]:
# make it so that each year of each leader's term is a row
df['Year'] = [pd.date_range(x, y,freq='YS',inclusive='both') for x, y in zip(df.start_year, df.end_year)]
df = df.explode('Year')
df['Year'] = df['Year'].astype(str).apply(lambda x:x[:4])

In [27]:
# read in geojson for choropleth 
with open('data/countries.geojson') as f:
    gj = geojson.load(f)

# add in the 'id' which allows plotly to associate the cordinates to a teh countries in my dataframe
for feature in gj['features']:
    feature['id'] = feature['properties']['ISO_A3']

In [ ]:
# only do three years to save memory b/c choropleth is to large otherwise
df_less = df[['Name','Country','Office','Head of state or government','Alpha-3 code','Year']].sort_values('Year')
df_less = df_less[df_less['Year'].isin([str(year) for year in range(2020,2023)])]

### Choropleth visualzation

In [ ]:
# without animation (only 2022)
fig = px.choropleth_mapbox(df[[df_less]==2022], geojson=gj, locations='Alpha-3 code', 
                    color='Head of state or government', mapbox_style='carto-positron', zoom=1,
                           color_discrete_sequence=px.colors.qualitative.Prism,
                          hover_data=['Name','Country','Office'])

fig.show()

In [45]:
# with animation (2020 - 2023)
fig = px.choropleth_mapbox(df_less, geojson=gj, locations='Alpha-3 code', 
                    color='Head of state or government', mapbox_style='carto-positron', zoom=1,
                           color_discrete_sequence=px.colors.qualitative.Prism,
                          hover_data=['Name','Country','Office'],animation_frame='Year')

fig.show()

### Correlation between percent women in congress and democracy index